In [1]:
#BROKER = "mqtt.eclipseprojects.io"
#TOPIC = "gw01/duese03"

#BROKER = "broker.hivemq.com"
#TOPIC = "gw01/duese02"

BROKER = "kleve.cool"
TOPIC = "gw/duese002"
USER = "pub"
PW = "pub"

import paho.mqtt.client as mqtt
import numpy as np
#from numpy import zeros
import bqplot.pyplot as plt
from bqplot import LinearScale
import bqplot
from IPython.display import display, clear_output
import ipywidgets as widgets

In [2]:
out = widgets.Output(layout={'border': '1px solid black'})
out.append_stdout('HERE THE DECODED MQTT MESSAGES WILL BE SHOWN!\n')
display(out)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [3]:
N = 200
x_data = np.array(np.zeros(N))  # Initialize with empty arrays
z_data = np.array(np.zeros(N))  # Initialize with empty arrays
t_data = np.array(np.zeros(N))  # Initialize with empty arrays

t = 0
x0 = 1200
z0 = 470

fig = plt.figure(title='Plant Profile Scanner')

x_scale = LinearScale(min=200, max=1000)  # Set the x-axis limits
y_scale = LinearScale(min=0, max=250)  # Set the y-axis limits

samples = plt.scatter(x=x_data, y=z_data,  colors=['red'], marker='circle', default_size=100, default_opacities=[1.0], display_legend=False, scales={'x': x_scale, 'y': y_scale})

text_widget = plt.label([f"{t = :8.2f} "], x=[10], y=[250], colors=['black'])

display(fig)

i = 0
t0 = traw = t = x = z = 0
payload = ""
#first = True

def on_message(client, userdata, message):
    global payload
    global i, t, t0, x, z, traw
    global t_data, x_data, z_data

    # Parse the received message
    payload = message.payload.decode('utf-8')
    traw, xraw, zraw = map(float, payload.strip('()').split(','))

    if (i==0): 
        t0 = traw

    t = traw - t0
    x = x0 - xraw
    z = z0 - zraw
    
    s = f"{i:4d}  {t:12.2f}  {x:6.2f}  {z:6.2f}"

#    out.append_stdout(s + "\n")

    i += 1

    t_data = np.roll(t_data, -1); t_data[-1] = t    
    x_data = np.roll(x_data, -1); x_data[-1] = x
    z_data = np.roll(z_data, -1); z_data[-1] = z

    # # index of x_acc array with the x closest to x_acc[idx]
    # idx = int(np.round(x/dx)) 
    # if (idx >= 0) & (idx < N_acc):
    #     z_acc[idx] = (n_acc[idx] * z_acc[idx] + z) / (n_acc[idx] + 1)
    #     n_acc[idx] += 1
     
    # # Update the line plot with new data
    with samples.hold_sync(): 
        samples.x, samples.y = x_data, z_data

    # # Update the text widget with the latest 't' value
    text_widget.text = [f"{t = :8.2f} s"]

    # Refresh the plot
    #clear_output(wait=True)
    #display(fig)


# MQTT setup
def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))
    out.append_stdout("Connected with result code " + str(rc) + "\n")

client = mqtt.Client()

client.on_connect = on_connect
client.on_message = on_message

client.username_pw_set(USER, PW)
client.connect(BROKER, 1883, 60)

client.subscribe(TOPIC)

#while (client.is_connected == False): pass
#out.append_stdout("MQTT Client connected.\n")

# Start the MQTT client loop in the background
client.loop_start()

# Display the figure
#display(fig)


Figure(axes=[Axis(scale=LinearScale(max=1000.0, min=200.0)), Axis(orientation='vertical', scale=LinearScale(ma…

In [4]:
#samples.x, samples.y = x_acc, z_acc

In [5]:
#z_acc

In [6]:
#profile.x, profile.y = x_acc, z_acc

In [7]:
#len(x_acc)

In [8]:
#len(z_acc)

In [9]:
#len(n_acc)

In [10]:
#n_acc

In [11]:
#z_acc

In [12]:
#z_data